In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import *
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

In [0]:
"""
Status
       1       then 'Email Sent'
       5 or 9  then 'SMS Sent'

Campaign Type
       0 then 'Undefined' 
	   1 then 'Newsletter' 
	   2 then 'Service'
	   3 then 'Sales'
	   4 then 'Transactional' 
	   5 then 'Loyalty' 
	   6 then 'Acquisition'
	   7 then 'Win-back/Anti-churn'
	   8 then 'Welcome/Onboarding' 

Mechanism
       0 then 'Undefined' 
	   1 then 'Triggered' 
	   2 then 'Automated Scheduled'
	   3 then 'Manual Send Out'
       
       
 Type (TrackingLog)
       2 then 'Opens'
       1 the 'Clicks'
"""

Out[ 2 ]: "\nStatus\n 1 then 'Email Sent'\n 5 or 9 then 'SMS Sent'\n\nCampaign Type\n 0 then 'Undefined' \n\t 1 then 'Newsletter' \n\t 2 then 'Service'\n\t 3 then 'Sales'\n\t 4 then 'Transactional' \n\t 5 then 'Loyalty' \n\t 6 then 'Acquisition'\n\t 7 then 'Win-back/Anti-churn'\n\t 8 then 'Welcome/Onboarding' \n\nMechanism\n 0 then 'Undefined' \n\t 1 then 'Triggered' \n\t 2 then 'Automated Scheduled'\n\t 3 then 'Manual Send Out'\n \n \n Type (TrackingLog)\n 2 then 'Opens'\n 1 the 'Clicks'\n"

In [0]:
current_date                        = datetime.datetime.today()
destination_table                   = "data_acm.acmdata_databrickscopy_all_countries"


# Campaign Data Overview
export_timestamp                    = date.today().strftime('%Y%m%d')  #datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
year_startdate                      = '2018-12-31 23:59:59'
destination_table_overview          = "data_acm.campaign_data_overview_based_on_databricks_copy_of_acm"


dbfs_mount_path                     = "dbfs:/mnt/menaka"
csv_folderpath                      = "%s/campaign_data_overview/" % dbfs_mount_path
csv_filename                        = "campaign_data_overview_based_on_databricks_copy_of_acm_" + export_timestamp
print(csv_filename)

delta_period                        = 6 #months
history_datetime                    = current_date - relativedelta(months= delta_period)
print(history_datetime)

campaign_data_overview_based_on_databricks_copy_of_acm_20190917
2019-03-17 08:58:40.097247

In [0]:
df = spark.sql("""
SELECT CASE WHEN delivery.CountryCode IS NULL THEN recipient.countryCode ELSE delivery.CountryCode END AS CountryCode,
       broadlog.broadlogId, 
       broadlog.deliveryId, 
       broadlog.RecipientId, 
       recipient.UniqueContactIdentifier,
       recipient.OHUBOperatorID,     
       delivery.CampaignType, 
       delivery.Mechanism,    
       delivery.DeliveryCode as deliveryName, 
       delivery.Label as waveName,
       CASE WHEN broadlog.EventDate IS NOT NULL THEN from_unixtime(unix_timestamp(broadlog.EventDate, 'dd-MM-yyyy HH:mm:ss'),'yyyy-MM-dd HH:mm:ss') END as sendDate,
   
       SUM(CASE WHEN url.Type = 2 THEN 1 ELSE 0 END ) as NoofOpens,
       SUM(CASE WHEN url.Type = 1 THEN 1 ELSE 0 END ) as NoofClicks, 
       
       MIN(CASE WHEN url.Type = 2 THEN  from_unixtime(unix_timestamp(trackinglog.Date, 'dd-MM-yyyy HH:mm:ss'),'yyyy-MM-dd HH:mm:ss') END ) as Min_OpenDate,
       MAX(CASE WHEN url.Type = 2 THEN from_unixtime(unix_timestamp(trackinglog.Date, 'dd-MM-yyyy HH:mm:ss'),'yyyy-MM-dd HH:mm:ss') END ) as Max_OpenDate,
       
       MIN(CASE WHEN url.Type = 1 THEN from_unixtime(unix_timestamp(trackinglog.Date, 'dd-MM-yyyy HH:mm:ss'),'yyyy-MM-dd HH:mm:ss') END ) as Min_ClickDate,
       MAX(CASE WHEN url.Type = 1 THEN from_unixtime(unix_timestamp(trackinglog.Date, 'dd-MM-yyyy HH:mm:ss'),'yyyy-MM-dd HH:mm:ss') END ) as Max_ClickDate    
FROM data_user_abhijeet.ods_ac_broadlogrcp broadlog
LEFT JOIN data_user_abhijeet.ods_ac_delivery delivery ON delivery.deliveryId = broadlog.DeliveryId
LEFT JOIN data_user_abhijeet.ods_ac_recipient recipient ON broadlog.RecipientId = recipient.RecipientId
LEFT JOIN data_user_abhijeet.ods_ac_trackinglogrcp trackinglog ON trackinglog.broadlogId = broadlog.broadlogId
LEFT JOIN data_user_abhijeet.ods_ac_trackingurl url ON url.TrackingUrlId = trackinglog.UrlId
WHERE broadlog.status = 1              -- Successfully Delivered
GROUP BY CASE WHEN delivery.CountryCode IS NULL THEN recipient.countryCode ELSE delivery.CountryCode END,
       broadlog.broadlogId, 
       broadlog.deliveryId, 
       broadlog.RecipientId, 
       recipient.UniqueContactIdentifier,
       recipient.OHUBOperatorID,            
       delivery.CampaignType, 
       delivery.Mechanism,   
       delivery.DeliveryCode,      
       delivery.Label ,       
       CASE WHEN broadlog.EventDate IS NOT NULL THEN from_unixtime(unix_timestamp(broadlog.EventDate, 'dd-MM-yyyy HH:mm:ss'),'yyyy-MM-dd HH:mm:ss') END
""")

In [0]:
df.write.mode("overwrite").saveAsTable(destination_table)

In [0]:
campaign_data               = spark.table(destination_table)

campaign_data_overview      = campaign_data.groupby("countryCode").agg(
  countDistinct(when(col("sendDate") > history_datetime, col("broadlogId"))).alias("NoofDeliveries_Last6Months"),
  countDistinct(when(col("sendDate") > history_datetime, col("recipientId"))).alias("NoofRecipients_Last6Months"),
  countDistinct(when(col("sendDate") > history_datetime, col("UniqueContactIdentifier"))).alias("NoofUniqueContactIdentifiers_Last6Months"),
  
  countDistinct(when( (col("CampaignType") == 1) & (col("sendDate") > year_startdate) , col("broadlogId"))).alias("NoofNewsletterDeliveries_In_2019"),           countDistinct(when( (col("CampaignType") == 1)   & (col("sendDate") > year_startdate) , col("recipientId"))).alias("NoofNewsletterRecipients_In_2019"),
  countDistinct(when( (col("CampaignType") == 1) & (col("sendDate") > year_startdate), col("UniqueContactIdentifier"))).alias("NoofNewsletterRecipients_OHUBIds_In_2019"),
  
  countDistinct(when( (col("CampaignType") == 1) & (col("sendDate") > history_datetime) , col("broadlogId"))).alias("NoofNewsletterDeliveries_Last6Months"),         countDistinct(when( (col("CampaignType") == 1) & (col("sendDate") > history_datetime) , col("recipientId"))).alias("NoofNewsletterRecipients_Last6Months"),
  countDistinct(when( (col("CampaignType") == 1) & (col("sendDate") > history_datetime), col("UniqueContactIdentifier"))).alias("NoofNewsletterRecipients_OHUBIds_Last6Months"), 
                                                                       
  countDistinct(col("broadlogId")).alias("NoofDeliveries"),                                                                      
  countDistinct(col("recipientId")).alias("NoofRecipients"),
  countDistinct(col("UniqueContactIdentifier")).alias("NoofUniqueContactIdentifiers"), 
  
  countDistinct(when(col("CampaignType") == 1 , col("broadlogId"))).alias("NoofNewsletterDeliveries"),            
  countDistinct(when(col("CampaignType") == 1 , col("recipientId"))).alias("NoofNewsletterRecipients"),
  countDistinct(when(col("CampaignType") == 1 , col("UniqueContactIdentifier"))).alias("NoofNewsletterRecipients_OHUBIds"), 
                                                                       
  countDistinct(when( (col("sendDate") > history_datetime) & (col("UniqueContactIdentifier").isNull()), col("broadlogId"))).alias("NoofDeliveries_UnMapped_toRcp"),   countDistinct(when( (col("deliveryName").isNull()) | (col("waveName").isNull()), col("broadlogId"))).alias("NoofDeliveries_MissingCampaignInfo"),   
  min(col("sendDate")).alias("Min_Date"), 
  max(col("sendDate")).alias("Max_Date"))

In [0]:
campaign_data_overview.write.mode("overwrite").saveAsTable(destination_table_overview)